In [6]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd
import glob

from sklearn.preprocessing import MinMaxScaler

from Indicator import Indicator

def fillindicator(df):
    indicator = Indicator()
    train_set = indicator.MACD(df,12,26,9,1)#MACD 1
    train_set = indicator.MACD(df,40,60,34,2)#MACD 2
    train_set = indicator.DMI(df,14)
    train_set = indicator.STOCHRSI(df)
    train_set = indicator.AO(df)
    train_set = indicator.WILLIANSR(df)
    train_set = indicator.RSI(df)
    train_set = indicator.MA(df,26)
    train_set['MA-26'] = train_set['MA']
    train_set = indicator.MA(df,15)
    train_set = indicator.check_trend(df)
    return train_set

def shiftdata(df,col_name,sft):
    for i in range(sft):
        i+=1
        shiftname = col_name+'-'+str(i)
        df[shiftname] = df[col_name].shift(i)

    return df

def norm(df,col_name):# this will get columnname + _n
    #Normalize data each colunm in to range -1 to 1
    df[col_name+'_n'] = 2*(df[col_name]-df[col_name].min())/(df[col_name].max()-df[col_name].min())-1    
    return df





In [7]:
df = pd.read_csv('AES.csv')

In [8]:
df = fillindicator(df)
df['AO-signal'] = np.where(df['AO'] > df['AO'].shift(1),1,0)
df.loc[df['Trend'] == 'Down', 'Trend'] = 0.1
df.loc[df['Trend'] == 'Up', 'Trend'] = 0.9 

#Delta
df['MACD-SL-1'] = df['MACD-1'] - df['SIGNAL LINE1']
df['MACD-SL-2'] = df['MACD-2'] - df['SIGNAL LINE2']
df['DMI'] = df['plusDI'] - df['minusDI']
df['STOCHRSI'] = df['%K'] - df['%D']
df['MA-trend'] = df['MA-26'] - df['Close']

#MACD
df = norm(df,'MACD-SL-1')
df = norm(df,'MACD-SL-2')
df = norm(df,'MACD-1')
df = norm(df,'MACD-2')
df = norm(df,'SIGNAL LINE1')
df = norm(df,'SIGNAL LINE2')

#DMI
df = norm(df,'plusDI')
df = norm(df,'minusDI')
df = norm(df,'DMI')

#StochasticRSI
df = norm(df,'%K')
df = norm(df,'%D')
df = norm(df,'STOCHRSI')

#Other
df = norm(df,'Close')
df = norm(df,'%R')
df = norm(df,'RSI')
df = norm(df,'AO')
df = norm(df,'MA-trend')

dataset = df[63:]
dataset.to_csv('Dataset.csv',index = False)

C:\Users\jame_\anaconda3\lib\site-packages\ta\trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
C:\Users\jame_\anaconda3\lib\site-packages\ta\trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])
